# Portfolio Backtesting with Turtle Trading System

This notebook demonstrates the new portfolio backtesting capabilities that allow you to:

- **Fixed Capital Management**: Start with $10,000 and maintain position limits
- **Signal Ranking**: Buy the top 10 ranked stocks according to strategy signals
- **Portfolio Rebalancing**: Hold stocks until exit signals, then buy next best signals
- **Performance Analytics**: Comprehensive analysis using quantstats-style metrics

## Key Features
- Multi-stock portfolio management
- Signal-based stock selection with ranking
- Configurable position sizing and limits
- Multiple exit strategies (ATR, EMA, MACD, Profit/Loss)
- Benchmark comparison (SPY, QQQ)
- Risk analytics and performance metrics

In [ ]:
import sys
sys.path.insert(0, "/home/jack/github/turtle-backtest")
print(sys.path)

from datetime import datetime
import pandas as pd
import logging

# Configure logging for better debugging
logging.basicConfig(level=logging.INFO)

from turtle.service.data_update_service import DataUpdateService
from turtle.common.enums import TimeFrameUnit
from turtle.portfolio import PortfolioBacktester, PortfolioAnalytics
from turtle.exit.atr import ATRExitStrategy

print("✅ Imports successful")

## Setup and Configuration

In [ ]:
# Initialize data service
data_service = DataUpdateService(time_frame_unit=TimeFrameUnit.DAY)

# Backtest parameters
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2024, 8, 30)
INITIAL_CAPITAL = 10000.0
MAX_POSITIONS = 10
POSITION_SIZE = 1000.0  # $1000 per position
MIN_SIGNAL_RANKING = 70  # Only consider signals ranked 70 or above

print("🎯 Backtest Configuration:")
print(f"   Period: {START_DATE.date()} to {END_DATE.date()}")
print(f"   Capital: ${INITIAL_CAPITAL:,.0f}")
print(f"   Max Positions: {MAX_POSITIONS}")
print(f"   Position Size: ${POSITION_SIZE:,.0f}")
print(f"   Min Signal Ranking: {MIN_SIGNAL_RANKING}")

## Define Stock Universe

You can use predefined symbol groups or create custom universes:

In [ ]:
# Option 1: Use predefined symbol groups
# symbol_group = data_service.symbol_group_repo.get_symbol_group_list('NAS100')
# universe = [x.symbol for x in symbol_group]

# Option 2: Custom universe for this example
universe = [
    # Technology
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'AVGO', 'NFLX', 'AMD', 'INTC',
    'CRM', 'ADBE', 'PYPL', 'ORCL', 'CSCO', 'TXN', 'QCOM', 'INTU', 'AMAT', 'ADI',
    # Healthcare & Biotech
    'UNH', 'JNJ', 'PFE', 'AMGN', 'GILD', 'MRNA', 'REGN', 'VRTX', 'BIIB', 'ISRG',
    # Consumer & Retail
    'TSLA', 'HD', 'MCD', 'NKE', 'SBUX', 'TGT', 'COST', 'WMT', 'DIS', 'NFLX',
    # Financial & Others
    'V', 'MA', 'JPM', 'BAC', 'GS', 'AXP', 'BRK.B', 'WFC', 'C', 'MS'
]

print(f"📊 Stock Universe: {len(universe)} stocks")
print(f"   Sample: {', '.join(universe[:10])}...")

## Choose Trading Strategy and Exit Strategy

Available strategies:
- **Trading Strategies**: Darvas Box, Mars, Momentum
- **Exit Strategies**: ATR, EMA, MACD, Profit/Loss

In [ ]:
# Choose trading strategy
# trading_strategy = data_service.darvas_box_strategy  # Trend-following
# trading_strategy = data_service.mars_strategy        # Mars momentum (@marsrides)
trading_strategy = data_service.momentum_strategy      # Traditional momentum

# Choose exit strategy
exit_strategy = ATRExitStrategy(
    bars_history=data_service.bars_history,
    time_frame_unit=TimeFrameUnit.DAY,
)

# Alternative exit strategies:
# exit_strategy = ProfitLossExitStrategy(
#     bars_history=data_service.bars_history,
#     time_frame_unit=TimeFrameUnit.DAY,
# )
#
# exit_strategy = EMAExitStrategy(
#     bars_history=data_service.bars_history,
#     time_frame_unit=TimeFrameUnit.DAY,
# )

print("🚀 Strategy Configuration:")
print(f"   Trading Strategy: {trading_strategy.__class__.__name__}")
print(f"   Exit Strategy: {exit_strategy.__class__.__name__}")

## Initialize Portfolio Backtester

In [ ]:
# Create portfolio backtester
backtester = PortfolioBacktester(
    trading_strategy=trading_strategy,
    exit_strategy=exit_strategy,
    bars_history=data_service.bars_history,
    initial_capital=INITIAL_CAPITAL,
    max_positions=MAX_POSITIONS,
    position_size=POSITION_SIZE,
    min_signal_ranking=MIN_SIGNAL_RANKING,
    time_frame_unit=TimeFrameUnit.DAY,
)

print("✅ Portfolio backtester initialized")

## Run Portfolio Backtest

This will:
1. Generate daily signals across the universe
2. Rank signals and select top performers
3. Manage position entries and exits
4. Track portfolio performance daily
5. Calculate comprehensive analytics

In [ ]:
print("🔄 Running portfolio backtest...")
print("This may take a few minutes depending on universe size and date range.")

# Run the backtest
results = backtester.run_backtest(
    start_date=START_DATE,
    end_date=END_DATE,
    universe=universe,
    benchmark_tickers=['SPY', 'QQQ']
)

print("✅ Backtest completed!")

## Performance Results

In [ ]:
# Display comprehensive performance summary
analytics = PortfolioAnalytics()
analytics.print_performance_summary(results)

## Detailed Analysis

In [ ]:
# Create DataFrame from closed positions for analysis
if results.closed_positions:
    trades_df = pd.DataFrame([
        {
            'ticker': pos.ticker,
            'entry_date': pos.entry_date,
            'exit_date': pos.exit_date,
            'entry_price': pos.entry_price,
            'exit_price': pos.exit_price,
            'shares': pos.shares,
            'realized_pnl': pos.realized_pnl,
            'realized_pnl_pct': pos.realized_pnl_pct,
            'holding_period_days': pos.holding_period_days,
            'exit_reason': pos.exit_reason,
            'entry_signal_ranking': pos.entry_signal_ranking
        }
        for pos in results.closed_positions
    ])

    print(f"📈 Trade Analysis: {len(trades_df)} completed trades")
    print("\nTrade Summary:")
    print(trades_df[['ticker', 'realized_pnl', 'realized_pnl_pct', 'holding_period_days']].describe())
else:
    print("⚠️  No completed trades found")
    trades_df = pd.DataFrame()

In [ ]:
# Performance by ticker
if not trades_df.empty:
    ticker_performance = trades_df.groupby('ticker').agg({
        'realized_pnl': ['count', 'sum', 'mean'],
        'realized_pnl_pct': 'mean',
        'holding_period_days': 'mean'
    }).round(2)

    ticker_performance.columns = ['Trade_Count', 'Total_PnL', 'Avg_PnL', 'Avg_Return_Pct', 'Avg_Days']
    ticker_performance = ticker_performance.sort_values('Total_PnL', ascending=False)

    print("\n📊 Performance by Ticker (Top 10):")
    print(ticker_performance.head(10))

In [ ]:
# Daily portfolio value chart
if not results.daily_values.empty:
    import matplotlib.pyplot as plt

    plt.figure(figsize=(12, 6))
    plt.plot(results.daily_values.index, results.daily_values.values, label='Portfolio Value', linewidth=2)
    plt.axhline(y=INITIAL_CAPITAL, color='gray', linestyle='--', alpha=0.7, label=f'Initial Capital (${INITIAL_CAPITAL:,.0f})')
    plt.title('Portfolio Value Over Time')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Daily returns distribution
    plt.figure(figsize=(10, 6))
    plt.hist(results.daily_returns.values, bins=50, alpha=0.7, edgecolor='black')
    plt.axvline(x=results.daily_returns.mean(), color='red', linestyle='--',
                label=f'Mean: {results.daily_returns.mean():.3f}%')
    plt.title('Daily Returns Distribution')
    plt.xlabel('Daily Return (%)')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  No daily value data available for charting")

## Trade Analysis Deep Dive

In [ ]:
# Best and worst trades
if not trades_df.empty:
    print("🏆 Top 5 Best Trades:")
    best_trades = trades_df.nlargest(5, 'realized_pnl_pct')[['ticker', 'entry_date', 'exit_date', 'realized_pnl', 'realized_pnl_pct', 'holding_period_days']]
    print(best_trades.to_string(index=False))

    print("\n💸 Top 5 Worst Trades:")
    worst_trades = trades_df.nsmallest(5, 'realized_pnl_pct')[['ticker', 'entry_date', 'exit_date', 'realized_pnl', 'realized_pnl_pct', 'holding_period_days']]
    print(worst_trades.to_string(index=False))

In [ ]:
# Exit reason analysis
if not trades_df.empty:
    print("\n🚪 Exit Reason Analysis:")
    exit_analysis = trades_df.groupby('exit_reason').agg({
        'realized_pnl': ['count', 'mean'],
        'realized_pnl_pct': 'mean',
        'holding_period_days': 'mean'
    }).round(2)

    exit_analysis.columns = ['Count', 'Avg_PnL_$', 'Avg_Return_%', 'Avg_Days']
    print(exit_analysis)

## Risk Analysis

In [ ]:
# Risk metrics breakdown
print("⚠️  Risk Analysis:")
print(f"   Maximum Drawdown: {results.max_drawdown_pct:.2f}%")
print(f"   Sharpe Ratio: {results.sharpe_ratio:.2f}")
print(f"   Volatility (Annualized): {results.volatility:.2f}%")

if results.benchmark_returns:
    print("\n📊 Benchmark Comparison:")
    portfolio_return = results.total_return_pct
    for ticker, benchmark_return in results.benchmark_returns.items():
        outperformance = portfolio_return - benchmark_return
        print(f"   {ticker}: {benchmark_return:.2f}% (Portfolio +{outperformance:.2f}%)")

## Experiment with Different Configurations

Try modifying these parameters and re-running the backtest:

```python
# Different position limits
MAX_POSITIONS = 5   # More concentrated portfolio
MAX_POSITIONS = 20  # More diversified portfolio

# Different signal thresholds
MIN_SIGNAL_RANKING = 80  # More selective (higher quality signals)
MIN_SIGNAL_RANKING = 60  # Less selective (more opportunities)

# Different position sizes
POSITION_SIZE = 500   # Smaller positions, more diversification
POSITION_SIZE = 2000  # Larger positions, more concentration

# Different time periods
START_DATE = datetime(2022, 1, 1)  # Different market conditions
END_DATE = datetime(2023, 12, 31)
```

## Enhanced Portfolio Analytics Features

This portfolio backtesting system now includes:

### ✅ **Comprehensive Performance Metrics**
- Traditional metrics: Sharpe, Sortino, Calmar ratios
- Advanced risk metrics: VaR, CVaR, Ulcer Index
- Distribution analysis: Skewness, Kurtosis, Tail Ratio  
- Trade statistics: Profit Factor, Gain-to-Pain Ratio

### ✅ **Professional Reporting**
- HTML tearsheet reports with quantstats integration
- Interactive charts and visualizations
- Monthly returns heatmaps
- Benchmark comparison analysis

### ✅ **Risk Management Analytics**
- Maximum drawdown analysis with recovery periods
- Rolling risk metrics and volatility analysis
- Value-at-Risk and Conditional Value-at-Risk calculations
- Comprehensive distribution and tail risk analysis

## Next Steps

1. **Optimize Parameters**: Use quantstats metrics to tune strategy parameters
2. **Benchmark Analysis**: Compare against different benchmarks (SPY, QQQ, sector ETFs)
3. **Sector Analysis**: Add sector-based diversification constraints
4. **Walk-Forward Testing**: Implement out-of-sample validation
5. **Risk Management**: Add position sizing based on volatility (ATR-based)
6. **Report Automation**: Generate regular tearsheet reports for strategy monitoring

This enhanced portfolio backtesting system provides institutional-quality analytics for systematic trading strategy development and validation.

In [ ]:
# Generate comprehensive QuantStats HTML report
output_file = "portfolio_tearsheet.html"
title = f"Portfolio Backtest Report - {START_DATE.strftime('%Y-%m-%d')} to {END_DATE.strftime('%Y-%m-%d')}"

html_report = analytics.create_quantstats_report(
    results=results,
    benchmark_ticker="SPY",
    output_file=output_file,
    title=title
)

if html_report:
    print(f"✅ QuantStats HTML report generated: {output_file}")
    print("   This comprehensive report includes:")
    print("   📈 Equity curve with drawdown visualization")
    print("   📊 Monthly returns heatmap")
    print("   📈 Return distribution analysis")
    print("   📉 Rolling statistics and risk metrics")
    print("   🎯 Benchmark comparison analysis")
    print("   📋 Complete performance metrics table")
    print(f"   🌐 Open {output_file} in your web browser to view the full report")
else:
    print("⚠️  Could not generate QuantStats report")

## Generate QuantStats HTML Report

Create a comprehensive HTML tearsheet report with professional-grade analytics and visualizations:

In [ ]:
# Get comprehensive quantstats metrics
qs_metrics = analytics.get_quantstats_metrics(results)

if qs_metrics:
    print("📊 QuantStats Comprehensive Metrics:")
    print(f"   CAGR: {qs_metrics['cagr']:.2f}%")
    print(f"   Volatility: {qs_metrics['volatility']:.2f}%")
    print(f"   Sharpe Ratio: {qs_metrics['sharpe_ratio']:.3f}")
    print(f"   Sortino Ratio: {qs_metrics['sortino_ratio']:.3f}")
    print(f"   Calmar Ratio: {qs_metrics['calmar_ratio']:.3f}")
    print(f"   Max Drawdown: {qs_metrics['max_drawdown']:.2f}%")
    print(f"   Value at Risk (95%): {qs_metrics['value_at_risk']:.2f}%")
    print(f"   Conditional VaR (95%): {qs_metrics['conditional_value_at_risk']:.2f}%")
    print(f"   Win Rate: {qs_metrics['win_rate']:.1f}%")
    print(f"   Profit Factor: {qs_metrics['profit_factor']:.2f}")
    print(f"   Gain-to-Pain Ratio: {qs_metrics['gain_to_pain_ratio']:.2f}")
    print(f"   Tail Ratio: {qs_metrics['tail_ratio']:.2f}")
    print(f"   Skewness: {qs_metrics['skewness']:.3f}")
    print(f"   Kurtosis: {qs_metrics['kurtosis']:.3f}")
    print(f"   Recovery Factor: {qs_metrics['recovery_factor']:.2f}")
    print(f"   Ulcer Index: {qs_metrics['ulcer_index']:.3f}")

    if 'best_month' in qs_metrics:
        print(f"   Best Month: {qs_metrics['best_month']:.2f}%")
        print(f"   Worst Month: {qs_metrics['worst_month']:.2f}%")
        print(f"   Average Win: {qs_metrics['avg_win']:.2f}%")
        print(f"   Average Loss: {qs_metrics['avg_loss']:.2f}%")
else:
    print("⚠️  QuantStats metrics not available (install quantstats package)")

## QuantStats Enhanced Analytics

The portfolio backtesting system now includes comprehensive quantstats integration for institutional-grade performance analysis.